In [2]:
import pandas as pd
import numpy as np
import unicodedata
import string
import re
import random
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from collections import Counter
import pickle
import random
import pdb
from torch.utils.data import DataLoader
import logging
import itertools
import unicodedata
import string
import re
import random
import argparse
from torch import optim
import time
import os
from torch.utils.data import BatchSampler
from torch.utils.data import SequentialSampler
from torch.utils.data import Sampler
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt
import math
import copy
import seaborn
import datetime
from logger import Logger

from torch.optim import RMSprop
import shutil
from statistics import mean

from gym_wrappers import MainGymWrapper
import gym

/home/cvh255/atari_game/logger.py:7: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/share/apps/python3/3.6.3/intel/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/share/apps/python3/3.6.3/intel/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/cvh255/pyenv/py3.6.3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/share/apps/python3/3.6.3/intel/lib/python3.6/site-packages/traitlets-4.3.2-py3.6.egg/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/cvh255/pyenv/py3.6.3/lib/python3.6/site-packages/ipykernel/kernelapp.py"

In [3]:
device = torch.device('cpu')

In [4]:
class ConvolutionalNeuralNetwork(nn.Module):
    def __init__(self, input_shape, action_space):
        super(ConvolutionalNeuralNetwork, self).__init__()
        self.input_shape = input_shape
        self.action_space = action_space
        self.conv1 = nn.Conv2d(in_channels=input_shape[0], out_channels=32, kernel_size=8, stride=(4,4))
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64,kernel_size=4,stride=(2,2))
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
        
        o_size = 3136
        self.linear1 = nn.Linear(in_features=o_size, out_features=512)
        self.linear2 = nn.Linear(in_features=512, out_features=self.action_space)
        
    def forward(self, x, batch_size=None):
        x = torch.from_numpy(x).float().to(device)
#         print(x.size(), torch.Size([1, 4, 84, 84]))
        out = F.relu(self.conv1(x))
        out = F.relu(self.conv2(out))
        out = F.relu(self.conv3(out))

        # out = (bs, nc, x, y)
        if batch_size==None:
            batch_size = out.size(0)
#         print(out.size())
        out_flat = out.view(batch_size, -1)
        
        out_flat = F.relu(self.linear1(out_flat))
        
        out_flat = self.linear2(out_flat)
        return out_flat

In [5]:
game_name = "Breakout-v0"

render = False
total_step_limit = 5000000
total_run_limit = 100
clip = True

FRAMES_IN_OBSERVATION = 4
FRAME_SIZE = 84
INPUT_SHAPE = (FRAMES_IN_OBSERVATION, FRAME_SIZE, FRAME_SIZE)

In [6]:
env_name = game_name + "Deterministic-v4" 
game_name, env_name

('Breakout-v0', 'Breakout-v0Deterministic-v4')

In [7]:
env = MainGymWrapper.wrap(gym.make(game_name))

In [8]:
GAMMA = 0.99
MEMORY_SIZE = 900000
BATCH_SIZE = 32
TRAINING_FREQUENCY = 4
TARGET_NETWORK_UPDATE_FREQUENCY = 40000
MODEL_PERSISTENCE_UPDATE_FREQUENCY = 10000
REPLAY_START_SIZE = 50000

EXPLORATION_MAX = 1.0
EXPLORATION_MIN = 0.1
EXPLORATION_TEST = 0.02
EXPLORATION_STEPS = 850000
EXPLORATION_DECAY = (EXPLORATION_MAX-EXPLORATION_MIN)/EXPLORATION_STEPS

In [9]:
class DDQNSolver:
    def __init__(self, game_name, input_shape, action_space, input_path):
        self.input_shape = input_shape
        self.game_name = game_name
        self.action_space = action_space
        self.input_path = input_path
        
        testing_model_path = input_path
        self.model_path = testing_model_path
        
        
        logger_path = "./output/logs/" + game_name + "/ddqn/testing/" + self._get_date() + "/"
        self.logger = Logger(game_name + " " + mode_name, logger_path)
        assert os.path.exists(os.path.dirname(testing_model_path)), "No testing model in: " + str(testing_model_path)

        self.ddqn = ConvolutionalNeuralNetwork(self.input_shape, action_space).to(device)

        if os.path.isfile(self.model_path):
            self.ddqn.load_state_dict(torch.load(testing_model_path))

#     def _save_model(self):
#         self.ddqn.save_weights(self.model_path)
    
#     def move(self, state):
#         if np.random.rand() < self.epsilon or len(self.memory) < REPLAY_START_SIZE:
#             return random.randrange(self.action_space)
        
#         q_values = self.ddqn(np.expand_dims(np.asarray(state).astype(np.float64), axis=0), batch_size=1)
#         return np.argmax(q_values[0].detach().cpu().numpy())
    
    def move(self, state):
        if np.random.rand() < EXPLORATION_TEST:
            return random.randrange(self.action_space)
        q_values = self.ddqn(np.expand_dims(np.asarray(state).astype(np.float64), axis=0), batch_size=1)
        return np.argmax(q_values[0].detach().cpu().numpy())
    
    def save_run(self, score, step, run):
        self.logger.add_score(score)
        self.logger.add_step(step)
        self.logger.add_run(run)

    def get_move(self, state):
        pass

    def remember(self, state, action, reward, next_state, done):
        pass

    def step_update(self, total_step):
        pass

    def _get_date(self):
        return str(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M'))

In [22]:
def main_loop(game_model, env, render, total_step_limit, total_run_limit, clip):
    run = 0
    total_step = 0
    max_score = 0
    while True:
        if run%10==0:
            print('-'*100)
            print("RUN = ",run)
        if total_run_limit is not None and run >= total_run_limit:
            print("Reached total run limit of: " + str(total_run_limit))
            print("Max score = ", max_score)
            return

        run += 1
        current_state = env.reset()
        step = 0
        score = 0
        
        start_game = time.time()
        while True:
            
            if total_step >= total_step_limit:
                print("Reached total step limit of: " + str(total_step_limit))
                exit(0)
            total_step += 1
            
            step += 1

            if render:
                env.render()

            action = game_model.move(current_state)
            next_state, reward, terminal, info = env.step(action)
            if clip:
                np.sign(reward)
            score += reward
            game_model.remember(current_state, action, reward, next_state, terminal)
            current_state = next_state

            game_model.step_update(total_step)

            if terminal:
                print("Time taken = ",time.time()-start_game, "for steps = ",step)
                if score>max_score:
                    max_score=score
                game_model.save_run(score, step, run)
                break

# Model Playing Atari using DDQN model

In [21]:
2.718*1000/954

2.849056603773585

In [17]:
3.26/1178

0.0027674023769100166

In [23]:
# path = "/home/cvh255/atari_game/output/neural_nets/Breakout-v0try4/actual_ddqn/2019-05-07/model.h5"
path = "/home/cvh255/atari_game/output/neural_nets/Breakout-v0try3/ddqn/2019-05-05/model.h5"
# game_mode, game_name, env.action_space.n
# game_mode,game_name, action_space
mode_name = "ddqn_testing"
game_mode = mode_name
game_model = DDQNSolver(game_name+"ddqn", INPUT_SHAPE, env.action_space.n, path)
main_loop(game_model, env, render, total_step_limit, total_run_limit, clip)

----------------------------------------------------------------------------------------------------
RUN =  0
Time taken =  4.798995494842529 for steps =  1707
Time taken =  5.397084474563599 for steps =  1950
Time taken =  2.479067087173462 for steps =  900
Time taken =  2.9815890789031982 for steps =  1072
Time taken =  4.439658880233765 for steps =  1598
Time taken =  1.9526028633117676 for steps =  704
Time taken =  1.5657103061676025 for steps =  568
Time taken =  3.5068626403808594 for steps =  1264
Time taken =  3.4364805221557617 for steps =  1247
Time taken =  2.3331451416015625 for steps =  843
score: (min: 13.0, avg: 31.5, max: 55.0
{"metric": "score", "value": 31.5}
step: (min: 568, avg: 1185.3, max: 1950
{"metric": "step", "value": 1185.3}
{"metric": "run", "value": 10}
----------------------------------------------------------------------------------------------------
RUN =  10
Time taken =  3.862818479537964 for steps =  1386
Time taken =  2.8435311317443848 for steps = 

In [24]:
path = "/home/cvh255/atari_game/output/neural_nets/Breakout-v0try4/actual_ddqn/2019-05-07/model.h5"
# game_mode, game_name, env.action_space.n
# game_mode,game_name, action_space
mode_name = "ddqn_testing"
game_mode = mode_name
game_model = DDQNSolver(game_name+"ddqn", INPUT_SHAPE, env.action_space.n, path)
main_loop(game_model, env, render, total_step_limit, total_run_limit, clip)

----------------------------------------------------------------------------------------------------
RUN =  0
Time taken =  4.053825616836548 for steps =  1412
Time taken =  3.461111307144165 for steps =  1229
Time taken =  5.41929030418396 for steps =  1926
Time taken =  2.960510730743408 for steps =  1045
Time taken =  3.460134267807007 for steps =  1226
Time taken =  4.300640344619751 for steps =  1525
Time taken =  5.3376617431640625 for steps =  1886
Time taken =  3.4642903804779053 for steps =  1230
Time taken =  4.39540696144104 for steps =  1570
Time taken =  5.016109228134155 for steps =  1782
score: (min: 10.0, avg: 42.6, max: 74.0
{"metric": "score", "value": 42.6}
step: (min: 1045, avg: 1483.1, max: 1926
{"metric": "step", "value": 1483.1}
{"metric": "run", "value": 10}
----------------------------------------------------------------------------------------------------
RUN =  10
Time taken =  3.9377400875091553 for steps =  1398
Time taken =  4.727349519729614 for steps =  

# Model Playing Atari using DQN model

In [11]:
# path = "/home/cvh255/atari_game/output/neural_nets/Breakout-v0try4/actual_ddqn/2019-05-07/model.h5"
path = "/home/cvh255/atari_game/output/neural_nets/Breakout-v0try3/ddqn/2019-05-05/model.h5"
# game_mode, game_name, env.action_space.n
# game_mode,game_name, action_space
mode_name = "ddqn_testing"
game_model = DDQNSolver(game_name+"dqn", INPUT_SHAPE, env.action_space.n, path)
main_loop(game_model, env, render, total_step_limit, total_run_limit, clip)

----------------------------------------------------------------------------------------------------
RUN =  0


/home/cvh255/pyenv/gym/gym/logger.py:30: UserWarning: WARN: <class 'gym_wrappers.NoopResetEnv'> doesn't implement 'step' method, which is required for wrappers derived directly from Wrapper. Deprecated default implementation is used.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


score: (min: 11.0, avg: 41.3, max: 59.0
{"metric": "score", "value": 41.3}
step: (min: 626, avg: 1659.6, max: 2987
{"metric": "step", "value": 1659.6}
{"metric": "run", "value": 10}
----------------------------------------------------------------------------------------------------
RUN =  10
score: (min: 12.0, avg: 30.4, max: 44.0
{"metric": "score", "value": 30.4}
step: (min: 929, avg: 1405.6, max: 2122
{"metric": "step", "value": 1405.6}
{"metric": "run", "value": 20}
----------------------------------------------------------------------------------------------------
RUN =  20
score: (min: 17.0, avg: 37.9, max: 61.0
{"metric": "score", "value": 37.9}
step: (min: 844, avg: 1442, max: 1784
{"metric": "step", "value": 1442}
{"metric": "run", "value": 30}
----------------------------------------------------------------------------------------------------
RUN =  30
score: (min: 16.0, avg: 42.7, max: 78.0
{"metric": "score", "value": 42.7}
step: (min: 873, avg: 1446.2, max: 2346
{"metric":

In [27]:
game_name

'Breakout-v0'

In [40]:
play(env)

TypeError: 'module' object is not callable

In [31]:
# from gym import play

In [34]:
from play import *

In [38]:
from gym.utils import play

/home/cvh255/pyenv/gym/gym/utils/play.py:7: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/share/apps/python3/3.6.3/intel/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/share/apps/python3/3.6.3/intel/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/cvh255/pyenv/py3.6.3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/share/apps/python3/3.6.3/intel/lib/python3.6/site-packages/traitlets-4.3.2-py3.6.egg/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/cvh255/pyenv/py3.6.3/lib/python3.6/site-packages/ipykernel/kernel